In [1]:
#Downloading CSVs
import pandas as pd
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
departments = pd.read_csv('departments.csv')
aisles = pd.read_csv('aisles.csv')


In [3]:
#Merging the CSVs
products = pd.merge(products, aisles, on='aisle_id', how='left')
products = pd.merge(products, departments, on='department_id', how='left')
orders_prior = pd.merge(order_products_prior, orders, on='order_id', how='left')
orders_full = pd.merge(orders_prior, products, on='product_id', how='left')


In [4]:
df = orders_full.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   product_id              int64  
 2   add_to_cart_order       int64  
 3   reordered               int64  
 4   user_id                 int64  
 5   eval_set                object 
 6   order_number            int64  
 7   order_dow               int64  
 8   order_hour_of_day       int64  
 9   days_since_prior_order  float64
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  aisle                   object 
 14  department              object 
dtypes: float64(1), int64(10), object(4)
memory usage: 3.6+ GB


In [6]:
df.isnull().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
department                      0
dtype: int64

In [7]:
df['days_since_prior_order'] = df['days_since_prior_order'].fillna(0)


In [8]:
df.isnull().sum()

order_id                  0
product_id                0
add_to_cart_order         0
reordered                 0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_name              0
aisle_id                  0
department_id             0
aisle                     0
department                0
dtype: int64

In [9]:
#Feature Engineering
user_features = df.groupby('user_id').agg(
    user_total_orders=('order_number', 'max'),
    user_avg_days_between_orders=('days_since_prior_order', 'mean'),
    user_days_since_last_order=('days_since_prior_order', 'last')
).reset_index()
product_features = df.groupby('product_id').agg(
    prod_total_orders=('reordered', 'count'),
    prod_reorders=('reordered', 'sum'),
    prod_reorder_ratio=('reordered', 'mean')
).reset_index()
user_product_features = df.groupby(['user_id', 'product_id']).agg(
    up_orders=('order_number', 'count'),
    up_first_order=('order_number', 'min'),
    up_last_order=('order_number', 'max'),
    up_reorders=('reordered', 'sum')
).reset_index()


final_features = user_product_features.merge(user_features, on='user_id', how='left').merge(product_features, on='product_id', how='left')

final_features['up_order_rate'] = final_features['up_orders'] / final_features['user_total_orders']


In [10]:
final_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 13 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   user_id                       int64  
 1   product_id                    int64  
 2   up_orders                     int64  
 3   up_first_order                int64  
 4   up_last_order                 int64  
 5   up_reorders                   int64  
 6   user_total_orders             int64  
 7   user_avg_days_between_orders  float64
 8   user_days_since_last_order    float64
 9   prod_total_orders             int64  
 10  prod_reorders                 int64  
 11  prod_reorder_ratio            float64
 12  up_order_rate                 float64
dtypes: float64(4), int64(9)
memory usage: 1.3 GB


In [11]:
product_info = df[['product_id', 'product_name', 'aisle', 'department']].drop_duplicates()

final_features_new = final_features.merge(product_info, on='product_id', how='left')


In [12]:
#Checking skewness and kurtosis
import pandas as pd

num_cols = final_features_new.select_dtypes(include=['int64', 'float64']).columns

skewness = final_features_new[num_cols].skew()


kurtosis = final_features_new[num_cols].kurtosis()

print("Skewness of numerical columns:")
print(skewness)

print("\nKurtosis of numerical columns:")
print(kurtosis)


Skewness of numerical columns:
user_id                         0.005730
product_id                     -0.033893
up_orders                       5.998498
up_first_order                  2.570427
up_last_order                   2.009479
up_reorders                     5.998498
user_total_orders               1.475029
user_avg_days_between_orders    0.494535
user_days_since_last_order      0.820885
prod_total_orders               5.158596
prod_reorders                   5.760321
prod_reorder_ratio             -0.685107
up_order_rate                   2.231538
dtype: float64

Kurtosis of numerical columns:
user_id                         -1.198117
product_id                      -1.173762
up_orders                       57.563038
up_first_order                   8.148618
up_last_order                    4.463504
up_reorders                     57.563038
user_total_orders                1.814197
user_avg_days_between_orders    -0.433890
user_days_since_last_order      -0.609926
prod_total_

In [13]:
# Log transforming
import numpy as np
log_transform_cols = [
    'up_orders', 'up_first_order', 'up_last_order', 
    'up_reorders', 'up_order_rate','user_total_orders', 'prod_total_orders', 'prod_reorders'
]

for col in log_transform_cols:
    final_features_new[col] = np.log1p(final_features_new[col])


In [14]:
import pandas as pd

num_cols = final_features_new.select_dtypes(include=['int64', 'float64']).columns

skewness = final_features_new[num_cols].skew()


kurtosis = final_features_new[num_cols].kurtosis()

print("Skewness of numerical columns:")
print(skewness)

print("\nKurtosis of numerical columns:")
print(kurtosis)


Skewness of numerical columns:
user_id                         0.005730
product_id                     -0.033893
up_orders                       1.923445
up_first_order                  0.413885
up_last_order                   0.116014
up_reorders                     1.515237
user_total_orders               0.014359
user_avg_days_between_orders    0.494535
user_days_since_last_order      0.820885
prod_total_orders              -0.216900
prod_reorders                  -0.294946
prod_reorder_ratio             -0.685107
up_order_rate                   1.752029
dtype: float64

Kurtosis of numerical columns:
user_id                        -1.198117
product_id                     -1.173762
up_orders                       3.864311
up_first_order                 -0.681974
up_last_order                  -0.783133
up_reorders                     1.842445
user_total_orders              -0.864572
user_avg_days_between_orders   -0.433890
user_days_since_last_order     -0.609926
prod_total_orders   

In [ ]:
#Joining with train CSV
order_products_train = pd.read_csv('D:\Product_Reordering\env\order_products__train.csv')
train_labels = order_products_train.merge(orders[['order_id', 'user_id']], on='order_id', how='left')

final_df = final_features_new.merge(train_labels[['user_id', 'product_id', 'reordered']], 
    on=['user_id', 'product_id'], 
    how='left'
)


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 17 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   user_id                       int64  
 1   product_id                    int64  
 2   up_orders                     float64
 3   up_first_order                float64
 4   up_last_order                 float64
 5   up_reorders                   float64
 6   user_total_orders             float64
 7   user_avg_days_between_orders  float64
 8   user_days_since_last_order    float64
 9   prod_total_orders             float64
 10  prod_reorders                 float64
 11  prod_reorder_ratio            float64
 12  up_order_rate                 float64
 13  product_name                  object 
 14  aisle                         object 
 15  department                    object 
 16  reordered                     float64
dtypes: float64(12), int64(2), object(3)
memory usage: 1.7+ GB


In [17]:
final_df.isnull().sum()

user_id                                0
product_id                             0
up_orders                              0
up_first_order                         0
up_last_order                          0
up_reorders                            0
user_total_orders                      0
user_avg_days_between_orders           0
user_days_since_last_order             0
prod_total_orders                      0
prod_reorders                          0
prod_reorder_ratio                     0
up_order_rate                          0
product_name                           0
aisle                                  0
department                             0
reordered                       12479129
dtype: int64

In [18]:
final_df['reordered'] = final_df['reordered'].fillna(0).astype(int)

In [19]:
final_df.isnull().sum()

user_id                         0
product_id                      0
up_orders                       0
up_first_order                  0
up_last_order                   0
up_reorders                     0
user_total_orders               0
user_avg_days_between_orders    0
user_days_since_last_order      0
prod_total_orders               0
prod_reorders                   0
prod_reorder_ratio              0
up_order_rate                   0
product_name                    0
aisle                           0
department                      0
reordered                       0
dtype: int64

In [21]:
id_cols = ['user_id', 'product_id', 'product_name', 'aisle', 'department']


In [ ]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
string_columns = final_df.columns[final_df.dtypes == 'object']
encoders = {}
for col in string_columns:
    le = LabelEncoder()
    final_df[col] = le.fit_transform(final_df[col])
    encoders[col] = le


In [23]:
x = final_df.drop('reordered', axis=1)
y = final_df['reordered']


In [24]:
#Under Sampling
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler(
    sampling_strategy=0.666, 
    random_state=42
)

x_under, y_under = undersampler.fit_resample(x, y)


In [25]:
#Train Test Split
from sklearn.model_selection import train_test_split


x_train_full, x_temp, y_train_full, y_temp = train_test_split(
    x_under, y_under, test_size=0.3, stratify=y_under, random_state=42
)

x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.67, stratify=y_temp, random_state=42
)


In [26]:
x_train_id = x_train_full[id_cols]
x_val_id = x_val[id_cols]
x_test_id = x_test[id_cols]

x_train_numeric = x_train_full.drop(columns=id_cols)
x_val_numeric = x_val.drop(columns=id_cols)
x_test_numeric = x_test.drop(columns=id_cols)


In [21]:
x_train_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1451312 entries, 6936249 to 9345974
Data columns (total 11 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   up_orders                     1451312 non-null  float64
 1   up_first_order                1451312 non-null  float64
 2   up_last_order                 1451312 non-null  float64
 3   up_reorders                   1451312 non-null  float64
 4   user_total_orders             1451312 non-null  float64
 5   user_avg_days_between_orders  1451312 non-null  float64
 6   user_days_since_last_order    1451312 non-null  float64
 7   prod_total_orders             1451312 non-null  float64
 8   prod_reorders                 1451312 non-null  float64
 9   prod_reorder_ratio            1451312 non-null  float64
 10  up_order_rate                 1451312 non-null  float64
dtypes: float64(11)
memory usage: 132.9 MB


In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_numeric)
x_val_scaled = scaler.transform(x_val_numeric)
x_test_scaled = scaler.transform(x_test_numeric)


In [ ]:
#Neural Network Model Creation and Compilation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam


model = Sequential()
model.add(Dense(256, input_dim=x_train_scaled.shape[1], activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Dense(1, activation="sigmoid")) 

model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy", Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]
)


early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,           
    restore_best_weights=True,
    verbose=1
)   


d:\GUVI\INSTACART\env\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
result = model.fit(
    x_train_scaled, y_train_full,
    validation_data=(x_val_scaled, y_val),
    epochs=50,
    batch_size=512,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
2835/2835 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.7282 - auc: 0.7891 - loss: 0.5467 - precision: 0.6743 - recall: 0.6212 - val_accuracy: 0.7446 - val_auc: 0.8123 - val_loss: 0.5158 - val_precision: 0.6978 - val_recall: 0.6368
Epoch 2/50
2835/2835 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.7458 - auc: 0.8115 - loss: 0.5173 - precision: 0.7013 - recall: 0.6351 - val_accuracy: 0.7438 - val_auc: 0.8120 - val_loss: 0.5161 - val_precision: 0.7031 - val_recall: 0.6214
Epoch 3/50
2835/2835 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.7453 - auc: 0.8113 - loss: 0.5175 - precision: 0.7017 - recall: 0.6327 - val_accuracy: 0.7433 - val_auc: 0.8122 - val_loss: 0.5172 - val_precision: 0.6942 - val_recall: 0.6396
Epoch 4/50
2835/2835 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.7464 - auc: 0.8122 - loss: 0.5164 - precision: 0.7028 - recall: 0.6345 - val_accuracy: 0.7446 - val_auc: 0.8135 - val_loss: 0.5146 - val_precision: 0.7120 - val_recall: 0.6067
Epoch 5/50
2835/2835

In [ ]:
#Saving the model
model.save('instacart_model1.keras')

In [ ]:
#Loading the model
from tensorflow import keras

model = keras.models.load_model('instacart_model1.keras')

In [ ]:
#Model Evaluation on Validation and Test Sets
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


y_val_pred_proba = model.predict(x_val_scaled)
y_val_pred = (y_val_pred_proba > 0.5).astype(int)

y_test_pred_proba = model.predict(x_test_scaled)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

print("VALIDATION METRICS")
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))
print("Validation AUC:", roc_auc_score(y_val, y_val_pred_proba))
print("Validation Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

print("TEST METRICS")
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))
print("Test AUC:", roc_auc_score(y_test, y_test_pred_proba))
print("Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


6415/6415 ━━━━━━━━━━━━━━━━━━━━ 5s 690us/step
13023/13023 ━━━━━━━━━━━━━━━━━━━━ 8s 649us/step
VALIDATION METRICS
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.79    123203
           1       0.70      0.64      0.67     82054

    accuracy                           0.75    205257
   macro avg       0.74      0.73      0.73    205257
weighted avg       0.74      0.75      0.74    205257

Validation AUC: 0.8142649659924064
Validation Confusion Matrix:
[[100567  22636]
 [ 29507  52547]]
TEST METRICS
Test Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.80    250141
           1       0.70      0.64      0.67    166594

    accuracy                           0.75    416735
   macro avg       0.74      0.73      0.73    416735
weighted avg       0.75      0.75      0.75    416735

Test AUC: 0.8157853607947017
Test Confusion Matrix:
[[204921 

In [15]:
x_test_decoded = x_test.copy()
for col in x_test_decoded.columns:
    if col in encoders:  
        x_test_decoded[col] = encoders[col].inverse_transform(x_test[col].astype(int))



In [17]:
test_results = x_test_decoded.copy()
test_results['user_id'] = x_test_id['user_id']
test_results['product_id'] = x_test_id['product_id']

test_results['actual_reordered'] = y_test
test_results['predicted_reordered'] = y_test_pred.flatten()
test_results['reorder_probability'] = y_test_pred_proba.flatten()


In [ ]:
#Business Use Case Analysis
print("\n1. PERSONALIZED PRODUCT RECOMMENDATION")
print("-" * 40)

high_reorder_products = test_results[test_results['reorder_probability'] > 0.7]
product_recommendations = high_reorder_products.groupby('product_name').agg({
    'reorder_probability': ['mean', 'count'],
    'actual_reordered': 'mean'
}).round(3)

product_recommendations.columns = ['avg_reorder_prob', 'frequency', 'actual_reorder_rate']
product_recommendations = product_recommendations.sort_values('avg_reorder_prob', ascending=False)

print("Top 10 products recommended for reorder:")
print(product_recommendations.head(10))

recommendation_accuracy = (high_reorder_products['actual_reordered'] == 1).mean()
print(f"\nRecommendation Accuracy (prob > 0.7): {recommendation_accuracy:.3f}")



1. PERSONALIZED PRODUCT RECOMMENDATION
----------------------------------------
Top 10 products recommended for reorder:
                                                    avg_reorder_prob  \
product_name                                                           
Lemon Tulsi Tea                                                0.914   
0 Calorie Strawberry Dragonfruit Water Beverage                0.913   
Jordan Almonds                                                 0.912   
Spreadable Butter With Canola Oil                              0.912   
Pink Beans                                                     0.910   
Chocolate Chip Muffins                                         0.910   
Yogurt, Coconut Non-Dairy, Low Fat, Vanilla                    0.909   
Energy Drink, Natural                                          0.909   
Carmelized Onion Hummus                                        0.909   
Peter Rabbit Organic Pea Spinach & Apple Puree ...             0.909   

             

In [23]:

print("\n2. INVENTORY MANAGEMENT")
print("-" * 30)


demand_forecast = test_results.groupby(['department', 'aisle']).agg({
    'reorder_probability': 'sum',
    'actual_reordered': 'sum',
    'product_name': 'count'
}).round(2)

demand_forecast.columns = ['predicted_demand', 'actual_demand', 'total_products']
demand_forecast['demand_accuracy'] = (
    1 - abs(demand_forecast['predicted_demand'] - demand_forecast['actual_demand']) / 
    demand_forecast['actual_demand'].clip(lower=1)
).round(3)

print("Demand forecast by department/aisle (top 10):")
print(demand_forecast.sort_values('predicted_demand', ascending=False).head(10))

overall_demand_accuracy = demand_forecast['demand_accuracy'].mean()
print(f"\nOverall Demand Forecasting Accuracy: {overall_demand_accuracy:.3f}")



2. INVENTORY MANAGEMENT
------------------------------
Demand forecast by department/aisle (top 10):
                                          predicted_demand  actual_demand  \
department aisle                                                            
produce    fresh fruits                       22259.039062          22337   
           fresh vegetables                   18395.609375          18300   
           packaged vegetables fruits         10242.509766          10503   
dairy eggs yogurt                              7738.069824           7588   
beverages  water seltzer sparkling water       5416.569824           5480   
dairy eggs milk                                5177.620117           5162   
           packaged cheese                     4916.979980           4935   
snacks     chips pretzels                      3704.379883           3789   
dairy eggs soy lactosefree                     3564.129883           3522   
bakery     bread                               3236

In [24]:
print("\n3. CUSTOMER RETENTION")
print("-" * 25)

if 'user_id' in test_results.columns:
    customer_analysis = test_results.groupby('user_id').agg({
        'reorder_probability': 'mean',
        'actual_reordered': 'mean',
        'product_name': 'count'
    }).round(3)
    
    customer_analysis.columns = ['avg_reorder_prob', 'actual_reorder_rate', 'products_purchased']
    
    high_churn_risk = customer_analysis[customer_analysis['avg_reorder_prob'] < 0.3]
    
    print(f"Customers at high churn risk: {len(high_churn_risk)}")
    print(f"Total customers: {len(customer_analysis)}")
    print(f"Churn risk rate: {len(high_churn_risk)/len(customer_analysis):.3f}")
    
    churn_prediction_accuracy = (
        (customer_analysis['avg_reorder_prob'] < 0.3) == 
        (customer_analysis['actual_reorder_rate'] < 0.3)
    ).mean()
    print(f"Churn prediction accuracy: {churn_prediction_accuracy:.3f}")
else:
    print("Note: user_id not found in data. Cannot perform customer-level churn analysis.")



3. CUSTOMER RETENTION
-------------------------
Customers at high churn risk: 47217
Total customers: 148623
Churn risk rate: 0.318
Churn prediction accuracy: 0.684


In [27]:
print("\n4. MARKETING OPTIMIZATION")
print("-" * 30)

import matplotlib.pyplot as plt
def get_marketing_segment(prob):
    if prob >= 0.8:
        return 'High Intent (Upsell)'
    elif prob >= 0.5:
        return 'Medium Intent (Nurture)'
    elif prob >= 0.2:
        return 'Low Intent (Re-engage)'
    else:
        return 'Very Low Intent (Win-back)'

test_results['marketing_segment'] = test_results['reorder_probability'].apply(get_marketing_segment)

marketing_segments = test_results.groupby('marketing_segment').agg({
    'reorder_probability': 'mean',
    'actual_reordered': 'mean',
    'product_name': 'count'
}).round(3)

marketing_segments.columns = ['avg_predicted_prob', 'actual_conversion_rate', 'segment_size']

print("Marketing segments analysis:")
print(marketing_segments)

for segment in marketing_segments.index:
    segment_data = test_results[test_results['marketing_segment'] == segment]
    precision = (segment_data['actual_reordered'] == 1).mean()
    print(f"{segment}: Precision = {precision:.3f}")




4. MARKETING OPTIMIZATION
------------------------------
Marketing segments analysis:
                            avg_predicted_prob  actual_conversion_rate  \
marketing_segment                                                        
High Intent (Upsell)                     0.852                   0.857   
Low Intent (Re-engage)                   0.345                   0.337   
Medium Intent (Nurture)                  0.645                   0.647   
Very Low Intent (Win-back)               0.101                   0.105   

                            segment_size  
marketing_segment                         
High Intent (Upsell)               40424  
Low Intent (Re-engage)            137388  
Medium Intent (Nurture)           111791  
Very Low Intent (Win-back)        127132  
High Intent (Upsell): Precision = 0.857
Low Intent (Re-engage): Precision = 0.337
Medium Intent (Nurture): Precision = 0.647
Very Low Intent (Win-back): Precision = 0.105


In [18]:
import joblib
joblib.dump(scaler, 'scaler1.pkl')


['scaler1.pkl']